In [2]:
import pickle
import torch

# Specify the path to your pickle file
pickle_file_path = 'results/11-9-14:38:47.config.pkl'
    
# Load the pickle file
with open(pickle_file_path, 'rb') as file:
    config = pickle.load(file)

from CellPLM.model import OmicsFormer

model = OmicsFormer(**config)

path = '/data/civy/integration/11-9-14:42:40_with_token_test.pt'

model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [3]:
import scanpy as sc
from scipy import  sparse

import torch

data = sc.read_h5ad('2000.h5ad')
if 'platform' not in data.obs:
    data.obs['platform'] = 'scRNA-seq'
if 'Dataset' not in data.obs:
    data.obs['Dataset'] = data.obs['study']
if 'batch_label' in data.obs:
    data.obs['batch'] = data.obs['batch_label']
gene_list = data.var.index.to_list()
gene_dict = dict(zip(data.obs['Dataset'].unique(), [torch.ones(len(gene_list)).bool()] * data.obs['Dataset'].nunique()))
ndataset = data.obs['Dataset'].nunique()


from CellPLM.utils.data import XDict, clean_batches, balanced_partition, data_setup

seq_list, batch_list, batch_labels, order_list, dataset_list, coord_list, label_list = data_setup(data, return_sparse=False)

partitions = balanced_partition(batch_list, 1)

In [5]:
import torch.nn as nn 
from torch.nn import  functional as F

class new_encoder(nn.Module):
    def __init__(self, encoder):
        super(new_encoder, self).__init__()
        
        self.__dict__.update(encoder.__dict__)  
        

    def forward(self, x_dict, output_attentions=False):
        h = x_dict['h']
        att_list = []
        for l in range(self.num_layers):

            if l == 0:
                x_dict['base0'] = h.detach()
            if output_attentions:
                h, att = self.layers[l](h, output_attentions=True)
                #print(att.shape)
                att_list.append(att)
            else:
                h = self.layers[l](h)
            if l == 0:
                x_dict['base1'] = h.detach()

        if False:
            return h
        else:
            return h

class Igwrapper(nn.Module):
    def __init__(self, model):
        super(Igwrapper, self).__init__()
        self.model = model
        self.encoder = new_encoder(model.encoder)
    def forward(self, x, batch, coord, gene_mask, dataset):
        x_dict = XDict({'x_seq': x.cuda(),  # seq_list[i].cuda(rank),
                        'batch': batch.cuda(),
                        'coord': coord.cuda(),
                        'gene_mask': gene_mask.cuda(),
                         'dataset': dataset.cuda(),
                        })
        x_dict['h'] = self.model.embedder(x_dict, None)
        if self.model.reg_token is not None:
            x_dict['h'] = torch.cat([x_dict['h'], self.model.reg_token])
        x_dict['h'] = self.encoder(x_dict, False)
        x_dict['h'] = x_dict['h'][:-8]
 
        x_dict['h'], latent_loss = self.model.latent(x_dict)
        if self.model.head_type is not None:
            out_dict, loss = self.model.head(x_dict)
            out_dict['latent_loss'] = latent_loss.item() if torch.is_tensor(latent_loss) else latent_loss
            out_dict['target_loss'] = loss.item()
        else:
            out_dict = self.model.decoder(x_dict)
        return out_dict['recon']
    def get_rep(self, x_dict):
        x_dict['h'] = self.model.embedder(x_dict, None)
        if self.model.reg_token is not None:
            x_dict['h'] = torch.cat([x_dict['h'], self.model.reg_token])
        x_dict['h'] = self.encoder(x_dict, False)
        return  x_dict['h'][-8:]
        


import torch.nn as nn 
from torch.nn import  functional as F

class Encoderwrapper(nn.Module):
    def __init__(self, model):
        super(Encoderwrapper, self).__init__()
        self.model = model
    def forward(self, x, batch, coord, gene_mask, dataset):
        x_dict = XDict({'x_seq': x.cuda(),  # seq_list[i].cuda(rank),
                        'batch': batch.cuda(),
                        'coord': coord.cuda(),
                        'gene_mask': gene_mask.cuda(),
                         'dataset': dataset.cuda(),
                        })
        x_dict['h'] = self.model.embedder(x_dict, None)
        if self.model.reg_token is not None:
            x_dict['h'] = torch.cat([x_dict['h'], self.model.reg_token])
        x_dict['h'] = self.model.encoder(x_dict, False)['hidden']
        #print(x_dict['h'].shape)
        if self.model.reg_token is not None:
            x_dict['h'] = x_dict['h'][:-8]
        x_dict['h'], latent_loss = self.model.latent(x_dict)
        return x_dict['h']

In [6]:
from CellPLM.utils.eval import minimum_eval, clustering_eval, CountCorr
import pandas as pd
import anndata as ad  
from captum.attr import  IntegratedGradients
from lfxai.explanations.features import attribute_auxiliary
from torch.utils.data import Dataset, TensorDataset, DataLoader

import hdf5plugin

c = []
model.eval()
model.cuda(0)

net = Igwrapper(model)

ig = IntegratedGradients(net)

attrs = []

num = 100
with torch.no_grad():
    for i in range(1):#len(batch_list)):
        c = []
        res = []
        attns = []
        index = torch.arange(seq_list[i].shape[0])[:num]
        x = seq_list[i][index]
        baseline = torch.zeros_like(x)
        attr = ig.attribute(inputs=x.cuda(0), target=2, baselines=baseline.cuda(0), additional_forward_args=(batch_list[i][:num].cuda(), coord_list[i][:num].cuda(), torch.ones([x.shape[1]]).bool().cuda(),  dataset_list[i][:num].cuda()))
        attrs.append(attr)

2023-12-17 12:33:41.598006: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/data/civy/miniconda3/lib/python3.10/site-packages/captum/attr/_utils/visualization.py:15: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [7]:
attrs = torch.cat(attrs)

final = attrs.reshape(1,100,2000)

normalize_final = torch.nn.functional.normalize(final, dim=-1)

sum_final = normalize_final.sum(1)

print(data.var_names[torch.argsort(-abs(sum_final[0].squeeze(0)))[:5].cpu()].tolist(),  data.var_names[2])

['CCL5', 'PRF1', 'GNLY', 'KLRD1', 'GZMA'] AOC1
